In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import numpy as np
import plotly.express as px
import folium
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from folium import plugins
from sklearn.preprocessing import StandardScaler
from folium.plugins import HeatMap
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import os
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


2024-05-09 18:42:00.285288: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [5]:
total_data

,Unnamed: 0,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,isType__h,isType__t,isType__u,isSold__PI,isSold__S,isSold__SA,isSold__SP,isSold__VB,len_parks,parks_nearby_0_geometry_lat,parks_nearby_0_geometry_lng,parks_nearby_0_rating,parks_nearby_0_place_id,parks_nearby_0_user_ratings_total,parks_nearby_1_geometry_lat,parks_nearby_1_geometry_lng,parks_nearby_1_rating,parks_nearby_1_place_id,parks_nearby_1_user_ratings_total,parks_nearby_2_geometry_lat,parks_nearby_2_geometry_lng,parks_nearby_2_rating,parks_nearby_2_place_id,parks_nearby_2_user_ratings_total,parks_nearby_3_geometry_lat,parks_nearby_3_geometry_lng,parks_nearby_3_rating,parks_nearby_3_place_id,parks_nearby_3_user_ratings_total,parks_nearby_4_geometry_lat,parks_nearby_4_geometry_lng,parks_nearby_4_rating,parks_nearby_4_place_id,parks_nearby_4_user_ratings_total,len_hospital,hospital_nearby_0_geometry_lat,hospital_nearby_0_geometry_lng,hospital_nearby_0_rating,hospital_nearby_0_place_id,hospital_nearby_0_user_ratings_total,hospital_nearby_1_geometry_lat,hospital_nearby_1_geometry_lng,hospital_nearby_1_rating,hospital_nearby_1_place_id,hospital_nearby_1_user_ratings_total,hospital_nearby_2_geometry_lat,hospital_nearby_2_geometry_lng,hospital_nearby_2_rating,hospital_nearby_2_place_id,hospital_nearby_2_user_ratings_total,hospital_nearby_3_geometry_lat,hospital_nearby_3_geometry_lng,hospital_nearby_3_rating,hospital_nearby_3_place_id,hospital_nearby_3_user_ratings_total,hospital_nearby_4_geometry_lat,hospital_nearby_4_geometry_lng,hospital_nearby_4_rating,hospital_nearby_4_place_id,hospital_nearby_4_user_ratings_total
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,1.474684,1.617195,2.0,1.0,1.0,1.004198,1.164335,9.474453,Yarra,-37.79960,144.99840,Northern Metropolitan,2.585440,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0,-37.797016,145.001133,4.6,ChIJ6W78oQhD1moREsJpTtb1TH0,326.0,-37.802150,145.003856,4.7,ChIJzbT6OAdD1moRR4sfmVP5dts,2117.0,-37.805396,145.000303,4.6,ChIJdUfF8gFD1moRFp4ahuX72rY,165.0,-37.798416,144.996831,4.7,ChIJUXAg55hD1moR5cvueQ1SbWY,55.0,-37.801478,144.996931,4.8,ChIJd7CzKKpD1moRxV0sNE7GWBo,10.0,4.0,-37.803451,144.994189,NaN,ChIJuWYvSx1D1moRzMLhuR3x9zE,NaN,-37.805532,144.991289,NaN,ChIJKSAKVuJC1moRlF9YfeJTgbE,NaN,-37.798778,144.987277,5.0,ChIJ0f4pK7VD1moRRJNDn-Ti-Bw,4.0,-37.798745,144.987213,5.0,ChIJaQREW55D1moRs9A3e9RCiHE,14.0,NaN,NaN,NaN,NaN,NaN
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,1.474684,1.617195,2.0,1.0,0.0,1.003242,1.104697,8.708029,Yarra,-37.80790,144.99340,Northern Metropolitan,2.585440,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0,-37.811093,144.987663,4.5,ChIJbXAIoOhC1moRddyfi3nHPX0,113.0,-37.805396,145.000303,4.6,ChIJdUfF8gFD1moRFp4ahuX72rY,165.0,-37.804712,144.984629,4.2,ChIJVzaVvuBC1moRThpBXc7-B70,129.0,-37.803444,144.994461,4.4,ChIJicwHtQJD1moRYIQxBXZWBA8,153.0,-37.810853,144.995745,5.0,ChIJk9TcWSpD1moRZs7H1iJGMWM,1.0,20.0,-37.812063,144.984121,4.0,ChIJb9HgMOhC1moRcfoF66cw_Xg,119.0,-37.814621,144.998825,2.4,ChIJm2UPHvpC1moRJiBcORsm290,184.0,-37.805532,144.991289,NaN,ChIJKSAKVuJC1moRlF9YfeJTgbE,NaN,-37.809711,144.995514,4.6,ChIJxdcSYPtC1moRRul4s6MAHwI,279.0,-37.803451,144.994189,NaN,ChIJuWYvSx1D1moRzMLhuR3x9zE,NaN
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,1.474684,1.617195,3.0,2.0,0.0,1.002785,1.198793,8.708029,Yarra,-37.80930,144.99440,Northern Metropolitan,2.585440,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,20.0,-37.811093,144.987663,4.5,ChIJbXAIoOhC1moRddyfi3nHPX0,113.0,-37.805396,145.000303,4.6,ChIJdUfF8gFD1moRFp4ahuX72rY,165.0,-37.810853,144.995745,5.0,ChIJk9TcWSpD1moRZs7H1iJGMWM,1.0,-37.804023,144.994362,4.0,ChIJ63wKhKBD1moR7xRZSUYdTQo,3.0,-37.803444,144.994461,4.4,ChIJicwHtQJD1moRYIQxBXZWBA8,153.0,20.0,-37.814621,144.998825,2.4,ChIJm2UPHvpC1moRJiBcORsm290,184.0,-37.812063,144.984121,4.0,ChIJb9HgMOhC1moRcfoF66cw_Xg,119.0,-37.817202,144.993061,3.6,ChIJPeyHE-5C1moRLxGQnd

In [8]:
#total_data = pd.read_csv('nodes_record_data.csv')
# split the DataFrame into 3 different subsets
train = total_data[total_data['train_mask'] == True].drop(columns=['train_mask', 'test_mask'])
test = total_data[total_data['test_mask'] == True].drop(columns=['train_mask', 'test_mask'])


KeyError: 'train_mask'

In [21]:
# Encoding the string dataframe (Regionname), (Council) and (Suburb) columns
ordinal_enc = OrdinalEncoder()
total_data['Encoded_Regionname'] = ordinal_enc.fit_transform(total_data[['Regionname']]).astype(int)
total_data['Encoded_Suburb'] = ordinal_enc.fit_transform(total_data[['Suburb']]).astype(int)
total_data['Encoded_Council'] = ordinal_enc.fit_transform(total_data[['CouncilArea']]).astype(int)
# Encoded Regionname Dictionary, I would use this part for analysing the dataset. As you can see below, the Regionname columns are encoded from 0 to 7.
Encoded_Regionname = total_data[["Regionname", "Encoded_Regionname"]].drop_duplicates().sort_values(by='Encoded_Regionname')
# Encoded Suburb Dictionary. As you can see below, the Suburb columns are encoded.
Encoded_Suburb = total_data[["Suburb", "Encoded_Suburb"]].drop_duplicates().sort_values(by='Encoded_Suburb')
# Encoded Council Dictionary. As you can see below, the Council columns are encoded.
Encoded_council = total_data[["CouncilArea", "Encoded_Council"]].drop_duplicates().sort_values(by='Encoded_Council')
suburb_data = total_data["Encoded_Suburb"].copy()
region_data = total_data["Encoded_Regionname"].copy()
council_data = total_data["Encoded_Council"].copy()


council_data = council_data.reset_index().rename(columns={"index":"node_id"})
suburb_data = suburb_data.reset_index().rename(columns={"index":"node_id"})
region_data = region_data.reset_index().rename(columns={"index":"node_id"})

council_data.to_csv('edges_council_data.csv')
suburb_data.to_csv('edges_suburb_data.csv')
region_data.to_csv('edges_region_data.csv')

Encoded_Suburb["Encoded_Suburb"].to_csv('nodes_suburb_data.csv')
Encoded_Regionname["Encoded_Regionname"].to_csv('nodes_region_data.csv')
Encoded_council["Encoded_Council"].to_csv('nodes_council_data.csv')

In [22]:
df_concated=pd.concat([total_data[["parks_nearby_0_place_id"]].rename(columns={"parks_nearby_0_place_id":"a"}),total_data[["parks_nearby_1_place_id"]].rename(columns={"parks_nearby_1_place_id":"a"}),total_data[["parks_nearby_2_place_id"]].rename(columns={"parks_nearby_2_place_id":"a"}),total_data[["parks_nearby_3_place_id"]].rename(columns={"parks_nearby_3_place_id":"a"}),total_data[["parks_nearby_4_place_id"]].rename(columns={"parks_nearby_4_place_id":"a"})],ignore_index=True)
ordinal_enc = OrdinalEncoder()
res = ordinal_enc.fit_transform(df_concated)
res = pd.DataFrame(res.astype(int))
len_res = int(len(res)/5)
res

,0
0,551
1,2931
2,2931
3,551
4,3081
...,...
66890,3140
66891,1596
66892,1072
66893,2899


In [23]:
total_data["parks_0_unique"] = res[:len_res]
total_data["parks_1_unique"] = res[len_res:len_res*2].reset_index().drop("index",axis=1)
total_data["parks_2_unique"] = res[len_res*2:len_res*3].reset_index().drop("index",axis=1)
total_data["parks_3_unique"] = res[len_res*3:len_res*4].reset_index().drop("index",axis=1)
total_data["parks_4_unique"] = res[len_res*4:len_res*5].reset_index().drop("index",axis=1)
Encoded_park_0 = total_data[["parks_nearby_0_geometry_lat","parks_nearby_0_geometry_lng","parks_nearby_0_rating", "parks_nearby_0_user_ratings_total", "parks_0_unique"]].drop_duplicates().sort_values(by='parks_0_unique')
Encoded_park_1 = total_data[["parks_nearby_1_geometry_lat","parks_nearby_1_geometry_lng","parks_nearby_1_rating", "parks_nearby_1_user_ratings_total", "parks_1_unique"]].drop_duplicates().sort_values(by='parks_1_unique')
Encoded_park_2 = total_data[["parks_nearby_2_geometry_lat","parks_nearby_2_geometry_lng","parks_nearby_2_rating", "parks_nearby_2_user_ratings_total", "parks_2_unique"]].drop_duplicates().sort_values(by='parks_2_unique')
Encoded_park_3 = total_data[["parks_nearby_3_geometry_lat","parks_nearby_3_geometry_lng","parks_nearby_3_rating", "parks_nearby_3_user_ratings_total", "parks_3_unique"]].drop_duplicates().sort_values(by='parks_3_unique')
Encoded_park_4 = total_data[["parks_nearby_4_geometry_lat","parks_nearby_4_geometry_lng","parks_nearby_4_rating", "parks_nearby_4_user_ratings_total", "parks_4_unique"]].drop_duplicates().sort_values(by='parks_4_unique')

In [24]:
Encoded_park_2

,parks_nearby_2_geometry_lat,parks_nearby_2_geometry_lng,parks_nearby_2_rating,parks_nearby_2_user_ratings_total,parks_2_unique
56,-38.071005,145.433433,4.3,10.0,-9223372036854775808
53,-37.768054,144.992696,5.0,1.0,-9223372036854775808
59,-37.725615,144.921552,NaN,NaN,-9223372036854775808
60,-37.730335,144.922869,5.0,1.0,-9223372036854775808
38,-37.775163,144.920678,4.4,29.0,-9223372036854775808
...,...,...,...,...,...
1556,-37.637626,145.091908,3.5,4.0,4731
1527,-37.907668,145.046885,4.7,24.0,4731
4309,-37.807307,144.951554,4.2,134.0,4733
4672,-37.839819,144.933724,NaN,NaN,4735


In [25]:
Encoded_park_0.drop(index=363,inplace=True)
Encoded_park_1.drop(index=363,inplace=True)
Encoded_park_2.drop(index=164,inplace=True)
Encoded_park_3.drop(index=164,inplace=True)
Encoded_park_4.drop(index=80,inplace=True)

In [26]:
# Filling the missing rating values with 0
Encoded_park_0["parks_nearby_0_rating"].fillna((0), inplace=True)
Encoded_park_0["parks_nearby_0_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_1["parks_nearby_1_rating"].fillna((0), inplace=True)
Encoded_park_1["parks_nearby_1_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_2["parks_nearby_2_rating"].fillna((0), inplace=True)
Encoded_park_2["parks_nearby_2_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_3["parks_nearby_3_rating"].fillna((0), inplace=True)
Encoded_park_3["parks_nearby_3_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_4["parks_nearby_4_rating"].fillna((0), inplace=True)
Encoded_park_4["parks_nearby_4_user_ratings_total"].fillna((0), inplace=True)
save0_train = Encoded_park_0.rename(columns={"parks_nearby_0_place_id":"a","parks_nearby_0_geometry_lng":"b","parks_nearby_0_rating":"c", "parks_nearby_0_user_ratings_total":"d", "parks_0_unique":"e","parks_nearby_0_geometry_lat":"f"}).copy()
save1_train = Encoded_park_1.rename(columns={"parks_nearby_1_place_id":"a","parks_nearby_1_geometry_lng":"b","parks_nearby_1_rating":"c", "parks_nearby_1_user_ratings_total":"d", "parks_1_unique":"e","parks_nearby_1_geometry_lat":"f"}).copy()
save2_train = Encoded_park_2.rename(columns={"parks_nearby_2_place_id":"a","parks_nearby_2_geometry_lng":"b","parks_nearby_2_rating":"c", "parks_nearby_2_user_ratings_total":"d", "parks_2_unique":"e","parks_nearby_2_geometry_lat":"f"}).copy()
save3_train = Encoded_park_3.rename(columns={"parks_nearby_3_place_id":"a","parks_nearby_3_geometry_lng":"b","parks_nearby_3_rating":"c", "parks_nearby_3_user_ratings_total":"d", "parks_3_unique":"e","parks_nearby_3_geometry_lat":"f"}).copy()
save4_train = Encoded_park_4.rename(columns={"parks_nearby_4_place_id":"a","parks_nearby_4_geometry_lng":"b","parks_nearby_4_rating":"c", "parks_nearby_4_user_ratings_total":"d", "parks_4_unique":"e","parks_nearby_4_geometry_lat":"f"}).copy()
saveall_df_train = pd.concat([save0_train,save1_train,save2_train,save3_train,save4_train], ignore_index=True).drop_duplicates(subset='e').sort_values(by='e')
saveall_df_train

,f,b,c,d,e
0,NaN,NaN,0.0,0.0,-9223372036854775808
15343,-37.820626,144.947855,4.0,4.0,0
11091,-37.710916,145.142843,0.0,0.0,1
3205,-37.702264,145.089274,3.7,10.0,2
3206,-37.700847,144.904671,0.0,0.0,3
...,...,...,...,...,...
19756,-37.576951,144.931762,0.0,0.0,4732
3201,-37.807307,144.951554,4.2,134.0,4733
6981,-37.822636,145.188380,4.0,3.0,4734
11068,-37.839819,144.933724,0.0,0.0,4735


In [27]:
saveall_df_train = pd.concat([save0_train,save1_train,save2_train,save3_train,save4_train], ignore_index=True).sort_values(by='e')
saveall_df_train

,f,b,c,d,e
0,NaN,NaN,0.0,0.0,-9223372036854775808
15329,-37.710614,144.995906,4.6,5.0,-9223372036854775808
15330,-37.728704,145.030477,0.0,0.0,-9223372036854775808
15331,NaN,NaN,0.0,0.0,-9223372036854775808
15332,-37.927792,145.176793,5.0,1.0,-9223372036854775808
...,...,...,...,...,...
19758,-37.839819,144.933724,0.0,0.0,4735
15319,-37.839819,144.933724,0.0,0.0,4735
15320,-37.802829,144.890895,0.0,0.0,4736
11069,-37.802829,144.890895,0.0,0.0,4736


In [28]:
scaler = MinMaxScaler(feature_range = (1,10))
saveall_df_train['f'] = scaler.fit_transform(saveall_df_train['f'].values.reshape(-1, 1))
saveall_df_train['b'] = scaler.fit_transform(saveall_df_train['b'].values.reshape(-1, 1))
saveall_df_train['d'] = scaler.fit_transform(saveall_df_train['d'].values.reshape(-1, 1))
# optional
# saveall_df_train['01'] = 0
# saveall_df_train['02'] = 0
# saveall_df_train['03'] = 0
# saveall_df_train['04'] = 0
# saveall_df_train['05'] = 0
# saveall_df_train['06'] = 0
# saveall_df_train['07'] = 0
# saveall_df_train['08'] = 0
# saveall_df_train['09'] = 0
# saveall_df_train['10'] = 0
# saveall_df_train['11'] = 0
# saveall_df_train['12'] = 0
# saveall_df_train['13'] = 0
# saveall_df_train['14'] = 0
# saveall_df_train['15'] = 0
# saveall_df_train['16'] = 0
# saveall_df_train['17'] = 0
# saveall_df_train['18'] = 0

saveall_df_train

,f,b,c,d,e
0,NaN,NaN,0.0,1.000000,-9223372036854775808
15329,6.514296,5.653403,4.6,1.002807,-9223372036854775808
15330,6.306105,5.936049,0.0,1.000000,-9223372036854775808
15331,NaN,NaN,0.0,1.000000,-9223372036854775808
15332,4.014921,7.132314,5.0,1.000561,-9223372036854775808
...,...,...,...,...,...
19758,5.027351,5.145017,0.0,1.000000,4735
15319,5.027351,5.145017,0.0,1.000000,4735
15320,5.453045,4.794845,0.0,1.000000,4736
11069,5.453045,4.794845,0.0,1.000000,4736


In [29]:
saveall_df_train['feat'] = saveall_df_train.drop("e",axis=1).astype(str).agg(', '.join, axis=1)
saveall_df_train = saveall_df_train[["e", "feat"]]
saveall_df_train

,e,feat
0,-9223372036854775808,"nan, nan, 0.0, 1.0"
15329,-9223372036854775808,"6.514295669964611, 5.653402508409499, 4.6, 1.0..."
15330,-9223372036854775808,"6.306105269887894, 5.9360494825084515, 0.0, 1.0"
15331,-9223372036854775808,"nan, nan, 0.0, 1.0"
15332,-9223372036854775808,"4.0149210236080535, 7.132314093799323, 5.0, 1...."
...,...,...
19758,4735,"5.0273506065884135, 5.145016661047748, 0.0, 1.0"
15319,4735,"5.0273506065884135, 5.145016661047748, 0.0, 1.0"
15320,4736,"5.453045154484187, 4.794845226807865, 0.0, 1.0"
11069,4736,"5.453045154484187, 4.794845226807865, 0.0, 1.0"


In [30]:
saveall_df_train.to_csv("nodes_parks_data.csv")

In [31]:
park_data_0 = total_data["parks_0_unique"].copy()
park_data_1 = total_data["parks_1_unique"].copy()
park_data_2 = total_data["parks_2_unique"].copy()
park_data_3 = total_data["parks_3_unique"].copy()
park_data_4 = total_data["parks_4_unique"].copy()

park_data_0 = park_data_0.reset_index().rename(columns={"index":"node_id"})
park_data_1 = park_data_1.reset_index().rename(columns={"index":"node_id"})
park_data_2 = park_data_2.reset_index().rename(columns={"index":"node_id"})
park_data_3 = park_data_3.reset_index().rename(columns={"index":"node_id"})
park_data_4 = park_data_4.reset_index().rename(columns={"index":"node_id"})

park_data_0 = park_data_0[park_data_0["parks_0_unique"]>=0]
park_data_1 = park_data_1[park_data_1["parks_1_unique"]>=0]
park_data_2 = park_data_2[park_data_2["parks_2_unique"]>=0]
park_data_3 = park_data_3[park_data_3["parks_3_unique"]>=0]
park_data_4 = park_data_4[park_data_4["parks_4_unique"]>=0]

park_data_0 = park_data_0.rename(columns={"parks_0_unique":"dest_id"}).copy()
park_data_1 = park_data_1.rename(columns={"parks_1_unique":"dest_id"}).copy()
park_data_2 = park_data_2.rename(columns={"parks_2_unique":"dest_id"}).copy()
park_data_3 = park_data_3.rename(columns={"parks_3_unique":"dest_id"}).copy()
park_data_4 = park_data_4.rename(columns={"parks_4_unique":"dest_id"}).copy()

saveall_df = pd.concat([park_data_0,park_data_1,park_data_2,park_data_3,park_data_4], ignore_index=True)
saveall_df

,node_id,dest_id
0,0,551
1,1,2931
2,2,2931
3,3,551
4,4,3081
...,...,...
66623,1832,491
66624,1833,1722
66625,1834,2660
66626,1835,1722


In [32]:
saveall_df.to_csv('edges_park_data.csv')
